### 1. Setup Inicial do Spark

In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import broadcast

from functools import reduce
import psycopg2

import sys
sys.path.append("../libs")
from utils import silenciar_warnings, pgconfig_init, pg_conn, pg_executar_sql, spark_init, spark_read_jdbc

# Silenciar warnings/logs
silenciar_warnings()

# Inicializar Spark
spark = spark_init("IBGE_Silver")

:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/hgirardi/.ivy2.5.2/cache
The jars for the packages stored in: /Users/hgirardi/.ivy2.5.2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-45d4330a-1fcd-4a3a-bc67-2197389cc6b8;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 50ms :: artifacts dl 1ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------

✅ Spark inicializado: IBGE_Silver [local[*]]


### 2. Setup Inicial do Postgres + Criação do schema Silver

In [8]:
# initizar configuração do PostgreSQL
pgConfig = pgconfig_init()

# criar schema silver no PostgreSQL, se não existir
with pg_conn(pgConfig) as conn:
    pg_executar_sql(conn, "CREATE SCHEMA IF NOT EXISTS silver;")

== Executando SQL no Postgres ==
✅ SQL executado!
== Processo finalizado ==


### 3. Buscar tabelas raw do schema Bronze + Manipulação inicial

In [3]:
# Buscar todas as tabelas
with pg_conn(pgConfig) as conn:
    tabelas = pg_executar_sql(conn, """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'bronze'
    """)

bronze_tables = [r[0] for r in tabelas] if tabelas is not None else []
print(f"✅ Tabelas encontradas no schema bronze: {bronze_tables}")


== Executando SQL no Postgres ==
✅ Query executada (4 linhas).
== Processo finalizado ==
✅ Tabelas encontradas no schema bronze: ['raw_dicionario_pnad_covid_112020_20220621', 'raw_pnad_covid_112020', 'raw_pnad_covid_092020', 'raw_pnad_covid_102020']


In [4]:
# Remover tabela de dicionário da lista
tabelas = [table for table in bronze_tables if 'dicionario' not in table.lower()]
dicionario = [table for table in bronze_tables if 'dicionario' in table.lower()]

print(f"Tabelas de dados: {tabelas}")
print(f"Tabelas de dicionário: {dicionario}")

Tabelas de dados: ['raw_pnad_covid_112020', 'raw_pnad_covid_092020', 'raw_pnad_covid_102020']
Tabelas de dicionário: ['raw_dicionario_pnad_covid_112020_20220621']


In [6]:
PARTES = 12          # paralelismo na leitura do JDBC

# Carregar todas as tabelas de dados em dataframes
dataframes = [spark_read_jdbc(spark, f"bronze.{t}", pgConfig, PARTES) for t in tabelas]

dataframes

[DataFrame[Ano: string, UF: string, CAPITAL: string, RM_RIDE: string, V1008: string, V1012: string, V1013: string, V1016: string, Estrato: string, UPA: string, V1022: string, V1023: string, V1030: string, V1031: string, V1032: string, posest: string, A001: string, A001A: string, A001B1: string, A001B2: string, A001B3: string, A002: string, A003: string, A004: string, A005: string, A006: string, A006A: string, A006B: string, A007: string, A007A: string, A008: string, A009: string, B0011: string, B0012: string, B0013: string, B0014: string, B0015: string, B0016: string, B0017: string, B0018: string, B0019: string, B00110: string, B00111: string, B00112: string, B00113: string, B002: string, B0031: string, B0032: string, B0033: string, B0034: string, B0035: string, B0036: string, B0037: string, B0041: string, B0042: string, B0043: string, B0044: string, B0045: string, B0046: string, B005: string, B006: string, B007: string, B008: string, B009A: string, B009B: string, B009C: string, B009D:

In [7]:

df_final = reduce(lambda a, b: a.unionByName(b, allowMissingColumns=True), dataframes)
df_final = df_final.repartition(PARTES * len(tabelas)).persist(StorageLevel.MEMORY_AND_DISK)

print("Registros Bronze (uma ação):", df_final.count())

Registros Bronze (uma ação): 1149197


In [10]:
# checar como os valores das três colunas adicionais de novembro (A006A, A006B e A007A) ficaram no df_final
# Confirmado que todos valores estão NULL
df_final \
    .select(
            F.col("Ano"),
            F.col("UF"),
            F.col("CAPITAL"),
            F.col("V1013"),
            F.col("A006A"),
            F.col("A006B"),
            F.col("A007A")
        ) \
    .filter(
        (F.col("V1013")=="10") &
        (F.col("A006A").isNotNull()) &
        (F.col("A006B").isNotNull()) &
        (F.col("A007A").isNotNull())
    ) \
    .show()

+---+---+-------+-----+-----+-----+-----+
|Ano| UF|CAPITAL|V1013|A006A|A006B|A007A|
+---+---+-------+-----+-----+-----+-----+
+---+---+-------+-----+-----+-----+-----+



### 4. Ajustar dicionário

In [11]:
# Carregamento do dicionário
print(f"tabela: bronze.{dicionario[0]}")
df_dicionario = spark.read.jdbc(pgConfig.jdbc_url, f"bronze.{dicionario[0]}", properties=pgConfig.jdbc_propriedades)
df_dicionario.persist(StorageLevel.MEMORY_AND_DISK)

df_dicionario.show(10)
df_dicionario.printSchema()

tabela: bronze.raw_dicionario_pnad_covid_112020_20220621


+-----+--------------------------------------+--------------------+----------+--------------------+----------+--------------------+
|index|Dicionário das variáveis da PNAD COVID|          Unnamed: 1|Unnamed: 2|          Unnamed: 3|Unnamed: 4|          Unnamed: 5|
+-----+--------------------------------------+--------------------+----------+--------------------+----------+--------------------+
|  110|                                   nan|                 nan|       nan|                 nan|        14|       Outro parente|
|  220|                                   nan|                 nan|       nan|                 nan|         3|            Não sabe|
|  165|                                   nan|                 nan|       nan|                 nan|         5|              5 dias|
|  605|                                   nan|                 nan|       nan|                 nan|         3|       Não solicitou|
|  495|                                   nan|                 nan|       na

In [12]:
# Analizar as perguntas no dicionário

df_dicionario.createOrReplaceTempView("bronze_dicionario")

query = """
select `Unnamed: 1` as codigo
     , `Unnamed: 2` as grupo
     , `Unnamed: 3` as pergunta
     , `index`
from bronze_dicionario
where `Unnamed: 3` != 'nan'
  and CAST(`index` as INTEGER) >= 96
order by CAST(`index` as INTEGER)
"""

perguntas_df = spark.sql(query)


In [13]:
perguntas_df.show(1000, truncate=False)

+-------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|codigo |grupo|pergunta                                                                                                                                                                      |index|
+-------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|A001   |A1   |Número de ordem                                                                                                                                                               |96   |
|A001A  |A1a  |Condição no domicílio                                                                                                                                                         |97   |
|A001B1 |A1b  |

#### 4.1 Selecionar perguntas

**Demografia:**
- UF
- V1022 - Situação do domicílio

**Dados temporais:**
- V1012 - Semana no mês
- V1013 - Mês da pesquisa

**Perguntas:**
- A002 - Idade do morador 
- A003 - Sexo
- A004 - Cor ou raça
- A005 - Escolaridade

- B008		O(A) Sr(a) fez algum teste para saber se estava infectado(a) pelo coronavírus? 
- B009A		Fez o exame coletado com cotonete na boca e/ou nariz (SWAB)? 
- B009B		Qual o resultado?
- B009C		Fez o exame de coleta de sangue através de furo no dedo?
- B009D		Qual o resultado?
- B009E		Fez o exame de coleta de sangue através da veia da braço?
- B009F		Qual o resultado?
- B011		Qual foi o resultado do teste?  Na semana passada, devido à pandemia do Coronavírus, em que medida o(a) Sr(a) restringiu o contato com as pessoas? 
        
- B0011		Na semana passada teve febre?  
- B0012		Na semana passada teve tosse?
- B0013		Na semana passada teve dor de garganta?
- B0014		Na semana passada teve dificuldade para respirar?
- B0015		Na semana passada teve dor de cabeça?
- B0016		Na semana passada teve dor no peito?
- B0017		Na semana passada teve náusea?
- B0018		Na semana passada teve nariz entupido ou escorrendo?
- B0019		Na semana passada teve fadiga?
- B00110		Na semana passada teve dor nos olhos?
- B00111		Na semana passada teve perda de cheiro ou sabor?
- B00112		Na semana passada teve dor muscular?
- B00113		Na semana passada teve diarreia?

- C013		Na semana passada, o(a) Sr(a) estava em trabalho remoto (home office ou teletrabalho)?
        

**Sócio/Econômico**
- C01011 - Número da faixa do rendimento/retirada em dinheiro
- D0051 - Auxílios emergenciais relacionados ao coronavirus        

In [14]:
# perguntas selecionadas + tradução
colunas_selecionadas = [
    
    # Demografia
    ("UF", "estado"),
    ("V1012", "semana_mes"),

    # Temporal
    ("V1013", "mes_pesquisa"),
    ("V1022", "situacao_domicilio"), 

    # Características pessoais
    ("A002", "idade"),
    ("A003", "sexo"),
    ("A004", "cor_raca"),
    ("A005", "escolaridade"),

    # Testes COVID
    ("B008", "fez_teste_covid"),
    ("B009A", "teste_swab"),
    ("B009B", "resultado_swab"),
    ("B009C", "teste_sangue_dedo"),
    ("B009D", "resultado_sangue_dedo"),
    ("B009E", "teste_sangue_braco"),
    ("B009F", "resultado_sangue_braco"),
    ("B011", "restricao_contato"),

    # Sintomas
    ("B0011", "febre"),
    ("B0012", "tosse"),
    ("B0013", "dor_garganta"),
    ("B0014", "dificuldade_respirar"),
    ("B0015", "dor_cabeca"),
    ("B0016", "dor_peito"),
    ("B0017", "nausea"),
    ("B0018", "nariz_entupido"),
    ("B0019", "fadiga"),
    ("B00110", "dor_olhos"),
    ("B00111", "perda_olfato_paladar"),
    ("B00112", "dor_muscular"),
    ("B00113", "diarreia"),

    # Trabalho
    ("C013", "trabalho_remoto"),

    # Socioeconômico
    ("C01011", "faixa_rendimento"),
    ("D0051", "auxilio_emergencial")
]

In [15]:
def analyze_unique_values(df, colunas_selecionadas):
    """
    Mostra valores únicos e contagens para cada coluna
    """
    
    df_temp = df.select(*[F.col(src).alias(dst) for src, dst in colunas_selecionadas])

    for nome_col, novo_nome in colunas_selecionadas:
        print(f"\n=== COLUNA: {novo_nome.upper()} ===")
        
        # Valores únicos com contagem
        unique_values = df_temp.groupBy(novo_nome).count().orderBy("count", ascending=False)
        total_unique = unique_values.count()
        
        print(f"Total de valores únicos: {total_unique}")
        
        # Mostrar os valores mais frequentes
        unique_values.show(1000, truncate=False)
        
        print("-" * 50)

In [16]:
# analizar o conteúdo de cada coluna para entender possíveis valores e ajustar a tipagem delas
analyze_unique_values(df_final, colunas_selecionadas)


=== COLUNA: ESTADO ===
Total de valores únicos: 27
+------+------+
|estado|count |
+------+------+
|31    |104901|
|35    |98621 |
|33    |93028 |
|43    |77959 |
|42    |67742 |
|21    |57260 |
|32    |52612 |
|29    |52150 |
|15    |50860 |
|41    |49680 |
|52    |49129 |
|27    |41701 |
|26    |39930 |
|23    |37365 |
|25    |35624 |
|51    |35480 |
|22    |32106 |
|24    |29079 |
|11    |27671 |
|28    |21456 |
|13    |18361 |
|14    |16886 |
|12    |16190 |
|50    |15246 |
|17    |12187 |
|53    |10489 |
|16    |5484  |
+------+------+

--------------------------------------------------

=== COLUNA: SEMANA_MES ===
Total de valores únicos: 4
+----------+------+
|semana_mes|count |
+----------+------+
|2         |292076|
|3         |287943|
|4         |285955|
|1         |283223|
+----------+------+

--------------------------------------------------

=== COLUNA: MES_PESQUISA ===
Total de valores únicos: 3
+------------+------+
|mes_pesquisa|count |
+------------+------+
|09       

#### 4.2 Ajustar DF de dicionário para melhor manipulação

In [17]:
def process_dictionary(df):
    """
    Processamento do dicionario para a criação de grupos dinâmicos para futura criação das tabelas de dimensão
    """
    window_spec = Window.orderBy("linha").rowsBetween(Window.unboundedPreceding, Window.currentRow)
    
    return (df
        .select(
            F.col("index").cast("integer").alias("linha"),
            F.col("`Unnamed: 1`").alias("variavel"),
            F.col("`Unnamed: 2`").alias("grupo"),
            F.col("`Unnamed: 3`").alias("descricao"),
            F.col("`Unnamed: 4`").alias("codigo_valor"),
            F.col("`Unnamed: 5`").alias("desc_valor")
        )

        # Pula cabeçalhos
        .filter(F.col("linha") > 2)
        
        # Só linhas com valores válidos
        .filter(
            (F.col("codigo_valor") != "nan") & F.col("codigo_valor").isNotNull() &
            (F.col("desc_valor") != "nan") & F.col("desc_valor").isNotNull()
        )
        .orderBy("linha")
        .withColumn("variavel", 
                   F.last(F.when(F.col("variavel") != "nan", F.col("variavel")), ignorenulls=True).over(window_spec))
        .withColumn("grupo", 
                   F.last(F.when(F.col("grupo") != "nan", F.col("grupo")), ignorenulls=True).over(window_spec))
        .withColumn("descricao", 
                   F.last(F.when(F.col("descricao") != "nan", F.col("descricao")), ignorenulls=True).over(window_spec))
        
        # Limpar quebras de linha de todas as colunas texto
        .withColumn("variavel", F.trim(F.regexp_replace("variavel", "\\n", " ")))
        .withColumn("grupo", F.trim(F.regexp_replace("grupo", "\\n", " ")))
        .withColumn("descricao", F.trim(F.regexp_replace("descricao", "\\n", " ")))
        .withColumn("codigo_valor", F.trim(F.regexp_replace("codigo_valor", "\\n", " ")))
        .withColumn("desc_valor", F.trim(F.regexp_replace("desc_valor", "\\n", " ")))
    )

In [18]:
df_dicionario_final = process_dictionary(df_dicionario)

In [19]:
df_dicionario_final.show(100, truncate=False)

+-----+--------+-----+-------------------------------------------------------------------------+----------------------------+--------------------------------------------------------------------------------------------+
|linha|variavel|grupo|descricao                                                                |codigo_valor                |desc_valor                                                                                  |
+-----+--------+-----+-------------------------------------------------------------------------+----------------------------+--------------------------------------------------------------------------------------------+
|4    |UF      |NULL |Unidade da Federação                                                     |11                          |Rondônia                                                                                    |
|5    |UF      |NULL |Unidade da Federação                                                     |12                          

**Integer**
- SEMANA_MES
- MES_PESQUISA
- IDADE

**Boolean**
- FEZ_TESTE_COVID
    - Mudar Ignorado para Não
    - Sim = True | Não = False 

- TESTE_SWAB | TESTE_SANGUE_DEDO | TESTE_SANGUE_BRACO | AUXILIO_EMERGENCIAL | TRABALHO_REMOTO
    - Mudar Ignorado para Null
    - Sim = True | Não = False 

In [20]:
def _to_bool(col, ignorado_as=None):
    """"
    Converte a coluna para booleano:
      - "sim" => True
      - "não" => False
      - "ignorado" => False ou NULL (depende do parâmetro)
    """
    v = F.lower(F.trim(col))
    if ignorado_as == "false":   # Ignorado => False
        return (F.when(v == "sim",  F.lit(True))
                 .when(v.isin("não", "ignorado"), F.lit(False))
                 .otherwise(F.lit(None)))
    else:                        # default: Ignorado => NULL
        return (F.when(v == "sim",  F.lit(True))
                 .when(v == "não", F.lit(False))
                 .when(v == "ignorado", F.lit(None))
                 .otherwise(F.lit(None)))


def create_silver_table(df_dados, df_dicionario, colunas_selecionadas):
    """ 
    Criação da tabela no schema silver, filtrando as perguntas (colunas)
    - Os códigos existentes no df_dados será traduzido com os valores no df_dicionario para facilitar a leitura
    - Colunas serão convertidas para os tipos corretos (inteiros/booleanos)
    - Devido ao tamanho da tabela, quantidade e simplicidade de dados, escolheu-se criar apenas uma tabela (sem o uso de tabelas dimensão e fato)
    """

    # selecione apenas as colunas desejadas
    cols_select = [c for c, _ in colunas_selecionadas]
    df_silver = df_dados.select(*cols_select)

    # dicionário preparado uma vez (var, cod, desc)
    df_dic = (df_dicionario
              .select(
                  F.col("variavel").alias("var"),
                  F.col("codigo_valor").cast("string").alias("cod"),
                  F.col("desc_valor").cast("string").alias("desc")
              ))

    # tradução dos códigos para os valores vindos do dicionário
    for col_cod, col_final in colunas_selecionadas:

        # lookup table
        lut = (df_dic
               .filter(F.col("var") == col_cod)
               .select(
                   F.col("cod").alias(col_cod),     # chave de junção
                   F.col("desc").alias("desc_temp")   # descrição final
               ))
        
        # join broadcast
        df_silver = (df_silver
                     .join(broadcast(lut), on=[col_cod], how="left")
                     .withColumn(col_final, F.coalesce(F.col("desc_temp"), F.col(col_cod)))
                     .drop("desc_temp", col_cod))

    # transformar colunas para integers
    for c in ["semana_mes", "mes_pesquisa", "idade"]: 
        df_silver = df_silver.withColumn(c, F.trim(F.col(c)).cast("int"))

    # transformar colunas para booleanos
    # regras:
    #   - fez_teste_covid: Ignorado => False
    #   - demais: Ignorado => NULL
    bool_false = "fez_teste_covid"
    bool_null  = ["teste_swab", "teste_sangue_dedo", "teste_sangue_braco",
                  "auxilio_emergencial", "trabalho_remoto"]

    df_silver = df_silver.withColumn(bool_false, _to_bool(F.col(bool_false), ignorado_as="false"))

    for c in bool_null:
        df_silver = df_silver.withColumn(c, _to_bool(F.col(c), ignorado_as="null"))

    return df_silver

In [21]:
df_silver_covid = create_silver_table(df_final, df_dicionario_final, colunas_selecionadas)
df_silver_covid = df_silver_covid.persist(StorageLevel.MEMORY_AND_DISK)

In [22]:
df_silver_covid.show(10)
df_silver_covid.printSchema()

+--------------+----------+------------+------------------+-----+------+--------+--------------------+---------------+----------+--------------+-----------------+---------------------+------------------+----------------------+--------------------+-----+-----+------------+--------------------+----------+---------+------+--------------+------+---------+--------------------+------------+--------+---------------+----------------+-------------------+
|        estado|semana_mes|mes_pesquisa|situacao_domicilio|idade|  sexo|cor_raca|        escolaridade|fez_teste_covid|teste_swab|resultado_swab|teste_sangue_dedo|resultado_sangue_dedo|teste_sangue_braco|resultado_sangue_braco|   restricao_contato|febre|tosse|dor_garganta|dificuldade_respirar|dor_cabeca|dor_peito|nausea|nariz_entupido|fadiga|dor_olhos|perda_olfato_paladar|dor_muscular|diarreia|trabalho_remoto|faixa_rendimento|auxilio_emergencial|
+--------------+----------+------------+------------------+-----+------+--------+-------------------

#### 4.3 Adição de colunas de controle para ajudar em futuras análises

##### 4.3.1 Colunas referente ao teste de COVID

In [23]:
# flag de testes realizados
# confirmado que os valores são True, False ou NULL para as três colunas

df_silver_covid.select(F.col("teste_swab")).distinct().show()
df_silver_covid.select(F.col("teste_sangue_dedo")).distinct().show()
df_silver_covid.select(F.col("teste_sangue_braco")).distinct().show()

+----------+
|teste_swab|
+----------+
|      true|
|     false|
|      NULL|
+----------+

+-----------------+
|teste_sangue_dedo|
+-----------------+
|             true|
|            false|
|             NULL|
+-----------------+

+------------------+
|teste_sangue_braco|
+------------------+
|              true|
|             false|
|              NULL|
+------------------+



In [24]:
df_testados = df_silver_covid.filter(
    (F.col("teste_swab") == True) |
    (F.col("teste_sangue_dedo") == True) |
    (F.col("teste_sangue_braco") == True)
)

# validar número de testes realizados
df_testados.select(F.count("*").alias("total_testes_realizados_true")).show()

# validar número de testados
df_silver_covid.filter(F.col("fez_teste_covid")==True).select(F.count("*").alias("total_testes_covid_true")).show()

# checar se existe algum valor NULL quando o teste foi realizado (True)
df_testados.filter(F.col("teste_swab")== True).select(F.col("resultado_swab")).distinct().show(truncate=False)
df_testados.filter(F.col("teste_sangue_dedo")== True).select(F.col("resultado_sangue_dedo")).distinct().show(truncate=False)
df_testados.filter(F.col("teste_sangue_braco")== True).select(F.col("resultado_sangue_braco")).distinct().show(truncate=False)

+----------------------------+
|total_testes_realizados_true|
+----------------------------+
|                      133400|
+----------------------------+

+-----------------------+
|total_testes_covid_true|
+-----------------------+
|                 135076|
+-----------------------+

+-----------------------------+
|resultado_swab               |
+-----------------------------+
|Ainda não recebeu o resultado|
|Ignorado                     |
|Inconclusivo                 |
|Negativo                     |
|Positivo                     |
+-----------------------------+

+-----------------------------+
|resultado_sangue_dedo        |
+-----------------------------+
|Ainda não recebeu o resultado|
|Inconclusivo                 |
|Negativo                     |
|Positivo                     |
|Ignorado                     |
+-----------------------------+

+-----------------------------+
|resultado_sangue_braco       |
+-----------------------------+
|Ainda não recebeu o resultado|
|Inconc

In [25]:
# confirmar se há alguma entrada onde mais de um tipo de teste seja True

df_testados \
    .filter(
        (F.col("teste_swab") == True) &
        (F.col("teste_sangue_dedo") == True)
    ) \
    .select(F.count("*").alias("total_testes_swab_sangue_dedo")) \
    .show()

df_testados \
    .filter(
        (F.col("teste_swab") == True) &
        (F.col("teste_sangue_braco") == True)
    ) \
    .select(F.count("*").alias("total_testes_swab_sangue_braco")) \
    .show()

df_testados \
    .filter(
        (F.col("teste_sangue_dedo") == True) &
        (F.col("teste_sangue_braco") == True)
    ) \
    .select(F.count("*").alias("total_testes_sangue_dedo_e_braco")) \
    .show()

df_testados \
    .filter(
        (F.col("teste_swab") == True) &
        (F.col("teste_sangue_dedo") == True) &
        (F.col("teste_sangue_braco") == True)
    ) \
    .select(F.count("*").alias("total_todos_testes")) \
    .show()

+-----------------------------+
|total_testes_swab_sangue_dedo|
+-----------------------------+
|                         8963|
+-----------------------------+

+------------------------------+
|total_testes_swab_sangue_braco|
+------------------------------+
|                          8184|
+------------------------------+

+--------------------------------+
|total_testes_sangue_dedo_e_braco|
+--------------------------------+
|                            6630|
+--------------------------------+

+------------------+
|total_todos_testes|
+------------------+
|              2653|
+------------------+



In [26]:
df_silver_covid.filter(
    (F.col("fez_teste_covid") == True) & 
    (F.col("teste_swab") != True) &
    (F.col("teste_sangue_dedo") != True) &
    (F.col("teste_sangue_braco") != True)
) \
.select(F.count("*").alias("fez_teste_covid_mas_tipo_teste_not_true")) \
.show()

# 133542 - 131785 = 1757

+---------------------------------------+
|fez_teste_covid_mas_tipo_teste_not_true|
+---------------------------------------+
|                                   1239|
+---------------------------------------+



In [27]:
df_silver_covid.filter(
    (F.col("fez_teste_covid") == False) & 
    (
        (F.col("teste_swab") == True) |
        (F.col("teste_sangue_dedo") == True) |
        (F.col("teste_sangue_braco") == True)
    )
    
) \
.select(F.count("*").alias("nao_fez_teste_covid_mas_tipo_teste_igual_true")) \
.show()


+---------------------------------------------+
|nao_fez_teste_covid_mas_tipo_teste_igual_true|
+---------------------------------------------+
|                                            0|
+---------------------------------------------+



In [28]:
resultados = ["resultado_swab", "resultado_sangue_dedo", "resultado_sangue_braco"]

# criar uma flag única para quem teve resultado = "Positivo" em qualquer uma das colunas acima
cond_resultados_positivo = reduce(
    lambda a, b: a | b,
    [F.coalesce(F.col(c) == F.lit("Positivo"), F.lit(False)) for c in resultados]
)

df_silver_covid = df_silver_covid.withColumn("teste_covid_positivo", cond_resultados_positivo)

In [29]:
# checar quem teve teste positivo, mas nenhum dos testes foi igual a Positivo
# expectativa é que não haja nenhum registro assim
df_silver_covid.filter(
    (F.col("teste_covid_positivo") == True) &
    (F.col("fez_teste_covid") == False)
).show(20, truncate=False)

+------+----------+------------+------------------+-----+----+--------+------------+---------------+----------+--------------+-----------------+---------------------+------------------+----------------------+-----------------+-----+-----+------------+--------------------+----------+---------+------+--------------+------+---------+--------------------+------------+--------+---------------+----------------+-------------------+--------------------+
|estado|semana_mes|mes_pesquisa|situacao_domicilio|idade|sexo|cor_raca|escolaridade|fez_teste_covid|teste_swab|resultado_swab|teste_sangue_dedo|resultado_sangue_dedo|teste_sangue_braco|resultado_sangue_braco|restricao_contato|febre|tosse|dor_garganta|dificuldade_respirar|dor_cabeca|dor_peito|nausea|nariz_entupido|fadiga|dor_olhos|perda_olfato_paladar|dor_muscular|diarreia|trabalho_remoto|faixa_rendimento|auxilio_emergencial|teste_covid_positivo|
+------+----------+------------+------------------+-----+----+--------+------------+---------------+

In [30]:
# checar quem teve teste positivo, mas nenhum dos testes foi igual a Positivo
# expectativa é que não haja nenhum registro assim
df_silver_covid.filter(
    (F.col("teste_covid_positivo") == True) &
    (F.col("resultado_swab") != "Positivo") &
    (F.col("resultado_sangue_dedo") != "Positivo") &
    (F.col("resultado_sangue_braco") != "Positivo")
).show(20, truncate=False)

+------+----------+------------+------------------+-----+----+--------+------------+---------------+----------+--------------+-----------------+---------------------+------------------+----------------------+-----------------+-----+-----+------------+--------------------+----------+---------+------+--------------+------+---------+--------------------+------------+--------+---------------+----------------+-------------------+--------------------+
|estado|semana_mes|mes_pesquisa|situacao_domicilio|idade|sexo|cor_raca|escolaridade|fez_teste_covid|teste_swab|resultado_swab|teste_sangue_dedo|resultado_sangue_dedo|teste_sangue_braco|resultado_sangue_braco|restricao_contato|febre|tosse|dor_garganta|dificuldade_respirar|dor_cabeca|dor_peito|nausea|nariz_entupido|fadiga|dor_olhos|perda_olfato_paladar|dor_muscular|diarreia|trabalho_remoto|faixa_rendimento|auxilio_emergencial|teste_covid_positivo|
+------+----------+------------+------------------+-----+----+--------+------------+---------------+

#### 4.3.2 - Colunas referente a sintomas

In [31]:
# Confirmar quais os valores existentes nas colunas de sintomas

sintomas = [
    "febre", "tosse", "dor_garganta", "dificuldade_respirar", "dor_cabeca",
    "dor_peito", "nausea", "nariz_entupido", "fadiga", "dor_olhos",
    "perda_olfato_paladar", "dor_muscular", "diarreia"
]

expressao = "stack({n}, {pairs})".format(
    n=len(sintomas),
    pairs=", ".join([f"'{c}', {c}" for c in sintomas])
)

df_sintomas = (df_silver_covid
    .selectExpr(f"{expressao} as (coluna, valor)")
    .cache()
)

df_sintomas.count()

for c in sintomas:
    (df_sintomas
        .filter(F.col("coluna") == c)
        .select(F.col("valor").alias(c))
        .distinct()
        .orderBy(c)
        .show(truncate=False)
    )

# confirmado que todas as colunas de sintomas possuem apenas quatro valores: Ignorado, Não, Não sabe e Sim. 

+--------+
|febre   |
+--------+
|Ignorado|
|Não     |
|Não sabe|
|Sim     |
+--------+

+--------+
|tosse   |
+--------+
|Ignorado|
|Não     |
|Não sabe|
|Sim     |
+--------+

+------------+
|dor_garganta|
+------------+
|Ignorado    |
|Não         |
|Não sabe    |
|Sim         |
+------------+

+--------------------+
|dificuldade_respirar|
+--------------------+
|Ignorado            |
|Não                 |
|Não sabe            |
|Sim                 |
+--------------------+

+----------+
|dor_cabeca|
+----------+
|Ignorado  |
|Não       |
|Não sabe  |
|Sim       |
+----------+

+---------+
|dor_peito|
+---------+
|Ignorado |
|Não      |
|Não sabe |
|Sim      |
+---------+

+--------+
|nausea  |
+--------+
|Ignorado|
|Não     |
|Não sabe|
|Sim     |
+--------+

+--------------+
|nariz_entupido|
+--------------+
|Ignorado      |
|Não           |
|Não sabe      |
|Sim           |
+--------------+

+--------+
|fadiga  |
+--------+
|Ignorado|
|Não     |
|Não sabe|
|Sim     |
+--------+


In [32]:
# criar uma flag única para quem respondeu "Sim" em qualquer um dos sintomas
condicao_sintomas_sim = reduce(
    lambda a, b: a | b,
    [F.coalesce(F.col(c) == F.lit("Sim"), F.lit(False)) for c in sintomas]
)

df_silver_covid = df_silver_covid.withColumn("possui_sintoma", condicao_sintomas_sim)

df_silver_covid.filter(F.col("possui_sintoma")==True).show(20, truncate=False)

+------------+----------+------------+------------------+-----+------+--------+------------------------------------+---------------+----------+-----------------------------+-----------------+---------------------+------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------+-----+-----+------------+--------------------+----------+---------+------+--------------+------+---------+--------------------+------------+--------+---------------+----------------+-------------------+--------------------+--------------+
|estado      |semana_mes|mes_pesquisa|situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |fez_teste_covid|teste_swab|resultado_swab               |teste_sangue_dedo|resultado_sangue_dedo|teste_sangue_braco|resultado_sangue_braco|restricao_contato                                                                                                         

### 5. Criar DF tabela Silver

In [33]:
from pyspark.sql.utils import AnalysisException
from py4j.protocol import Py4JJavaError

# Criação da tabela no Postgres
print(f"== Criação da tabela silver.covid no banco de dados ==")

nome_tabela = "silver.covid"
 
try:
    df_silver_covid.repartition(12, "estado", "semana_mes").write.mode("overwrite").jdbc(pgConfig.jdbc_url, nome_tabela, properties=pgConfig.jdbc_propriedades)
    print(f"  ✅ Tabela {nome_tabela} criada com sucesso!")

except Exception as e:
    print(f"  ❌ Erro ao escrever tabela silver.covid: {e}")
    
except AnalysisException as e:
    print(f"  ❌ Erro de análise Spark (arquivo/schema): {e}")
    
except psycopg2.Error as e:
    print(f"  ❌ Erro psycopg2: {e}")
    
except Py4JJavaError as e:
    print(f"  ❌ Erro Py4JJavaError: {e}")

except Exception as e:
    print(f"  ⚠️ Erro genérico: {e}")

finally:
    print("")
    print("== Criação das tabelas concluída == ")

== Criação da tabela silver.covid no banco de dados ==


  ✅ Tabela silver.covid criada com sucesso!

== Criação das tabelas concluída == 


In [34]:
# criação dos indexes para melhorar performance de consultas
print(f"== Criação dos índices na tabela silver.covid no banco de dados ==")

with pg_conn(pgConfig) as conn:
    pg_executar_sql(conn, """
        CREATE INDEX IF NOT EXISTS idx_silver_covid_estado ON silver.covid(estado);
        CREATE INDEX IF NOT EXISTS idx_silver_covid_mes ON silver.covid(mes_pesquisa);
        CREATE INDEX IF NOT EXISTS idx_silver_covid_semana_mes ON silver.covid(semana_mes);
        CREATE INDEX IF NOT EXISTS idx_silver_covid_idade ON silver.covid(idade);
        CREATE INDEX IF NOT EXISTS idx_silver_covid_fez_teste_covid ON silver.covid(fez_teste_covid);
        CREATE INDEX IF NOT EXISTS idx_silver_covid_possui_sintoma ON silver.covid(possui_sintoma);
    """)

== Criação dos índices na tabela silver.covid no banco de dados ==
== Executando SQL no Postgres ==
✅ SQL executado!
== Processo finalizado ==


In [35]:
df_silver_covid.show(20,truncate=False)
df_silver_covid.printSchema()

+--------------+----------+------------+------------------+-----+------+--------+------------------------------------+---------------+----------+--------------+-----------------+---------------------+------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------+-----+-----+------------+--------------------+----------+---------+------+--------------+------+---------+--------------------+------------+--------+---------------+----------------+-------------------+--------------------+--------------+
|estado        |semana_mes|mes_pesquisa|situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |fez_teste_covid|teste_swab|resultado_swab|teste_sangue_dedo|resultado_sangue_dedo|teste_sangue_braco|resultado_sangue_braco|restricao_contato                                                                                                               |febre|tosse|dor_gar